Use:
* https://github.com/inspirehep/magpie from the inpirehep records
* https://link.lens.org/ywt8QmB3gAh download
* https://link.lens.org/uQjoEFwtx1j download
See also: http://ai.stanford.edu/blog/introduction-to-knowledge-graphs/

In [37]:
import pandas as pd
import requests
import time
import sys

In [38]:
df=pd.read_json('https://raw.githubusercontent.com/restrepo/cms_pub/main/cms.json')

In [39]:
#df=df.drop('MA_topics',axis='columns')

* https://github.com/inspirehep/rest-api-doc

In order to avoid overwhelming the server, we enforce rate limits per IP address: every IP address is allowed 50 requests, then at most 2 requests per second. If you exceed those limits, you will receive a response with HTTP status code 429 and a x-retry-in header telling you how long to wait before retrying.

In [40]:
df[:1]

,CMS_id,report,title,journal,date,CTG,category,inspire_id,abstract,keywords,doi,MA_topics
0,58,FSQ-16-006,Study of central exclusive $\pi^{+}\pi^{-}$ pr...,EPJC 80 (2020) 718,2020-03-05,FSQ,Forward and Small-x QCD Physics,1784063,Central exclusive and semiexclusive production...,"[p p: scattering, p p: colliding beams, pi: pa...",10.1140/epjc/s10052-020-8166-5,"[Rapidity, Physics, Invariant mass, Nuclear ph..."


In [41]:
REBUILD=False
if not REBUILD:
    sys.exit('Go to next cell')
insprdf=pd.DataFrame()

#See: https://github.com/restrepo/PythonTipsAndTricks/blob/master/pandas/DataFrame.md#insert-list-into-a-cell
df['keywords']=''
df['keywords']=df['keywords'].astype(object)

SystemExit: Go to next cell

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [42]:
for i in df.index:
    if not REBUILD:
        print('Go to next cell')
        break
    print(i,end='\r')
    if not df.loc[i,"inspire_id"]:
        continue
    r=requests.get(f'https://inspirehep.net/api/literature/{df.loc[i,"inspire_id"]}')
    try:
        df.loc[i,'abstract']=r.json().get('metadata').get('abstracts')[0].get('value')
    except:
        df.loc[i,'abstract']=''
    try:
        df.at[i,'keywords']=[d.get('value') for d in r.json().get('metadata').get('keywords') 
                              if d.get('schema')=='INSPIRE' ]
    except:
        df.at[i,'keywords']=[]
    try:
        df.loc[i,'doi']=r.json().get('metadata').get('dois')[0].get('value')
    except:
        df.loc[i,'doi']=''

    try:
        insprdf=insprdf.append(r.json(),ignore_index=True).reset_index(drop=True)
    except:
        pass
    time.sleep(0.5)
    #break
insprdf=insprdf.rename({'id':'inspire_id'},axis='columns')    

Go to next cell


In [43]:
if REBUILD:
    df.to_csv('cms.csv',index=False)
    df.to_json('cms.json',orient='records')

In [44]:
if REBUILD:
    insprdf[['created', 'inspire_id', 'links', 'metadata', 'updated', 'uuid']].to_json('inspire.json',orient='records')

In [18]:
#insprdfdf=pd.read_json('inspire.json')

## Check keywords/MA Topics

In [49]:
if not REBUILD:
    sys.exit('Go to next cell')
ln=pd.read_json('MA_topics.json')

ln=ln[ln['doi']!=''].reset_index(drop=True)
ln['doi']=ln['doi'].str.lower()
ln=ln.rename({'doi':'ldoi'},axis='columns')

ln2=pd.read_csv('lens-lhc.csv',low_memory=False)
ln2['Fields of Study']=ln2['Fields of Study'].str.split('; ')
ln2=ln2[['DOI','Fields of Study']]
ln2=ln2[ln2['DOI']!=''].reset_index(drop=True)
ln2=ln2.dropna(subset=['DOI']).reset_index(drop=True)
ln2['DOI']=ln2['DOI'].str.lower()
ln2=ln2.rename({'DOI':'ldoi','Fields of Study':'MA_topics'},axis='columns')

ln.shape[0]+ln2.shape[0]

ln=ln.append(ln2).drop_duplicates('ldoi').reset_index(drop=True)

ln.shape

df['ldoi']=df['doi'].str.lower()

df=df.merge(ln,on='ldoi',how='left')

df=df.drop('ldoi',axis='columns')

SystemExit: Go to next cell

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [46]:
df['MA_topics'].shape

(1003,)

In [47]:
df['MA_topics'].dropna().shape

(747,)

In [50]:
df[:2]

,CMS_id,report,title,journal,date,CTG,category,inspire_id,abstract,keywords,doi,MA_topics
0,58,FSQ-16-006,Study of central exclusive $\pi^{+}\pi^{-}$ pr...,EPJC 80 (2020) 718,2020-03-05,FSQ,Forward and Small-x QCD Physics,1784063,Central exclusive and semiexclusive production...,"[p p: scattering, p p: colliding beams, pi: pa...",10.1140/epjc/s10052-020-8166-5,"[Rapidity, Physics, Invariant mass, Nuclear ph..."
1,57,FSQ-12-033,Measurement of single-diffractive dijet produc...,"EPJC 80, 1164 (2020)",2020-02-27,FSQ,Forward and Small-x QCD Physics,1782637,Measurements are presented of the single-diffr...,"[p p: colliding beams, parton: distribution fu...",10.1140/epjc/s10052-020-08562-y,None


In [17]:
#row='keywords'
row='MA_topics'

kk=df.copy()
    
if row=='keywords':
    kk[row]=kk[row].apply(lambda l: list(set([x.split(':')[0] for x in l])) if isinstance(l,list) else [])
kk=kk[kk[row].apply(lambda l: len(l) if isinstance(l,list) else 0)>0]
kk=kk.reset_index(drop=True)

In [18]:
kk.groupby('CTG')['CTG'].count().sort_values(ascending=False)

CTG
EXO    161
HIG    126
SMP    102
TOP     93
SUS     61
HIN     59
B2G     53
FSQ     48
BPH     44
Name: CTG, dtype: int64

In [19]:
kk['CTG'].unique()

array(['FSQ', 'BPH', 'SMP', 'TOP', 'HIG', 'SUS', 'EXO', 'B2G', 'HIN'],
      dtype=object)

In [20]:
unique_row=list(set(kk[row].sum()))

In [21]:
if row=='keywords':
    unique_row=list(set([x for x in unique_row if x.find('-->')==-1])) 

In [22]:
if row=='MA_topics':
    sus=['Minimal Supersymmetric Standard Model',  'Supersymmetry'
       ,'Chargino', 'Gaugino','Gluino','Goldstino','Gravitino','Lightest Supersymmetric Particle','Neutralino','R-parity'
         , 'Split supersymmetry','Stop squark','Superpartner',
     'Supersymmetry breaking'
    ]
    exo=['Dark matter',
     'Dark photon','Extra dimensions', 'Grand Unified Theory', 'Graviton','Hawking radiation','Hidden sector','Higgsino',
     'Kaluza–Klein theory','Leptoquark','Micro black hole', 'Physics beyond the Standard Model','Randall–Sundrum model','Seesaw mechanism',
         'Sterile neutrino','Technicolor','Two-Higgs-doublet model','Universal extra dimension','Unparticle physics','W′ and Z′ bosons','Large extra dimension'
    ]

    hig=['Higgs boson',
     'Higgs field',
     'Higgs mechanism',
     'Higgs sector', 'High mass','Search for the Higgs boson'
        ]

    sm=['Standard Model',
      'Gauge boson', 'W and Z bosons',
    'Cabibbo–Kobayashi–Maskawa matrix',
       ]

    qcd=['Quantum chromodynamics',
     'Gluon','B meson',
     'Bottom quark', 'J/psi meson','Pentaquark','Perturbative QCD','Tetraquark'
     ]
    
    hin=['Heavy ion','Quark–gluon plasma','Relativistic Heavy Ion Collider']

    b2g=['Third generation']

    top=['Top quark']

if row=='keywords':
    sus=[ 'supersymmetry', 'minimal supersymmetric standard model',
     'sneutrino',
     'gravitino',
     'Higgsino',
     'slepton',
     'sbottom',
     'stop',
    'R parity',
      'gaugino',
     'neutralino',
     'gluino',
     'chargino',
     'selectron',
    'bino',
      'sneutrino/tau',
    'smuon',
     'LSP',
     'squark',
     'stau']

    exo=[ 'little Higgs model', 'universal extra dimension', 'technicolor',
          "W'", 'leptoquark','graviton', 'seesaw model', 'unparticle',"Z'",'WIMP','hidden sector',
         'dark matter', 'Kaluza-Klein model','black hole', "top'", 'Randall-Sundrum model',
         'axion-like particles','WIMP nucleon']
    sm=[ 'standard model','CKM matrix']
    qcd=['quantum chromodynamics','gluon', 'tetraquark']

    hig=['Higgs particle'] 

    bph= ['B', 'bottom']
    top=['top'] 
    hin=['heavy ion']

In [23]:
kkk=kk[kk[row].apply(lambda l: list( set( sus ) & set(l) ) ).apply(len)>0]
kkk.groupby('CTG')['CTG'].count().sort_values(ascending=False)

CTG
SUS    61
EXO    36
HIG    19
B2G     5
TOP     3
SMP     1
Name: CTG, dtype: int64

In [24]:
kkk=kk[kk[row].apply(lambda l: list( set( exo ) & set(l) ) ).apply(len)>0]
kkk.groupby('CTG')['CTG'].count().sort_values(ascending=False)

CTG
EXO    81
SUS    21
B2G    17
HIG    15
TOP     5
SMP     4
BPH     1
Name: CTG, dtype: int64

In [25]:
kkk=kk[kk[row].apply(lambda l: list( set( hig ) & set(l) ) ).apply(len)>0]
kkk.groupby('CTG')['CTG'].count().sort_values(ascending=False)

CTG
HIG    119
B2G     22
EXO     20
SMP      7
TOP      6
SUS      6
FSQ      1
BPH      1
Name: CTG, dtype: int64

In [26]:
kkk=kk[kk[row].apply(lambda l: list( set( top ) & set(l) ) ).apply(len)>0]
kkk.groupby('CTG')['CTG'].count().sort_values(ascending=False)

CTG
TOP    77
B2G    28
HIG    17
SUS    12
EXO     9
HIN     1
Name: CTG, dtype: int64

In [27]:
kkk=kk[kk[row].apply(lambda l: list( set( hin ) & set(l) ) ).apply(len)>0]
kkk.groupby('CTG')['CTG'].count().sort_values(ascending=False)

CTG
HIN    34
FSQ     2
EXO     1
Name: CTG, dtype: int64

In [28]:
kkk=kk[kk[row].apply(lambda l: list( set( sm ) & set(l) ) ).apply(len)>0]
kkk.groupby('CTG')['CTG'].count().sort_values(ascending=False)

CTG
HIG    85
EXO    79
TOP    45
SMP    41
B2G    29
SUS    18
BPH     8
HIN     6
FSQ     3
Name: CTG, dtype: int64

In [29]:
kkk=kk[kk[row].apply(lambda l: list( set( qcd ) & set(l) ) ).apply(len)>0]
kkk.groupby('CTG')['CTG'].count().sort_values(ascending=False)

CTG
SMP    53
HIG    36
TOP    34
EXO    29
B2G    29
HIN    28
FSQ    17
BPH    11
SUS    10
Name: CTG, dtype: int64

https://link.lens.org/FoybzNctiBi

In [30]:
lf=kk[row].sum()
lfu=list(set( kk[row].sum() ))
pd.DataFrame(data=zip( lfu, [ lf.count(l) for l in lfu  ] ),columns=['key','count'] ).sort_values('count',ascending=False)[:50]

,key,count
375,Physics,763
157,Particle physics,672
145,Large Hadron Collider,640
409,Nuclear physics,493
81,Standard Model,302
374,Lepton,249
161,Quark,195
275,Muon,195
158,Luminosity (scattering theory),192
213,Higgs boson,184


In [52]:
%%writefile file.py
1+1

Writing file.py


In [53]:
cat file.py

1+1
